In [41]:
import akshare as ak
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)

code_list = ['sh000300', "sz399905"]
# code_list = ["sh000300"]

In [42]:
index_data = {}
for code in code_list:
    df = ak.stock_zh_index_daily(symbol=code)
    df["date"] = df["date"].apply(lambda x: x.strftime("%Y-%m-%d"))
    df = df[df["date"] >= "2016-01-01"]
    index_data[code] = {"code": code, "data": df}

In [43]:
for code in code_list:
    j = index_data[code]
    df = j["data"]
    close = df["close"].to_numpy()
    j["close"] = close
    j["open"] = df["open"].to_numpy()
    j["high"] = df["high"].to_numpy()
    j["low"] = df["low"].to_numpy()

In [44]:
dates = index_data["sh000300"]["data"]["date"].to_numpy()

In [56]:
# 单独测试RSI
account = 1000000.0
profits = []
cur_code = ""
amount = 0
prev_account = 0
default_hold_days = 1
hold_days = default_hold_days
fee = 0.001
for (index, date) in enumerate(dates):
    if index == 0:
        pass
    for code in code_list:
        # 如果今天的开盘低于昨天的收盘
        today_open = index_data[code]["open"][index]
        today_close = index_data[code]["close"][index]
        prev_low = index_data[code]["low"][index-1]
        today_high = index_data[code]["high"][index]

        if cur_code != "" and cur_code == code:
            hold_days -= 1
            if hold_days <= 0 :
                account = amount * today_close * (1 - fee)
                cur_code = ""
                amount = 0
                profit = account - prev_account
                print (date, "sell ", code, profit)
                profits.append(profit)
        if today_open < prev_low and today_open > prev_low * 0.995 and cur_code == "" and today_high > prev_low:
            cur_code = code
            amount = account / prev_low
            prev_account = account
            account = 0
            print (date, "buy ", code)
            hold_days = default_hold_days

if cur_code != "":
     account = amount * index_data[cur_code]["close"][-1]
     cur_code = ""
     amount = 0
     profit = account - prev_account
     print (date, "sell ", cur_code, profit)
     profits.append(profit)

2016-01-12 buy  sz399905
2016-01-13 sell  sz399905 -34284.49715357949
2016-01-26 buy  sh000300
2016-01-27 sell  sh000300 -55259.6499457676
2016-02-19 buy  sh000300
2016-02-22 sell  sh000300 19675.26980362204
2016-03-23 buy  sh000300
2016-03-24 sell  sh000300 -11575.996497645508
2016-03-25 buy  sh000300
2016-03-28 sell  sh000300 -4403.875590202282
2016-04-01 buy  sz399905
2016-04-05 sell  sz399905 21864.232607837417
2016-04-22 buy  sh000300
2016-04-25 sell  sh000300 2355.96612723195
2016-05-12 buy  sh000300
2016-05-13 sell  sh000300 2687.3061916129664
2016-05-16 buy  sh000300
2016-05-17 sell  sh000300 5603.310951962834
2016-05-20 buy  sh000300
2016-05-23 sell  sh000300 8045.557900175685
2016-05-30 buy  sz399905
2016-05-31 sell  sz399905 38254.57371094404
2016-06-02 buy  sh000300
2016-06-03 sell  sh000300 8270.554945341777
2016-06-13 buy  sz399905
2016-06-14 sell  sz399905 -36166.2449027159
2016-06-14 buy  sz399905
2016-06-15 sell  sz399905 33740.9075255451
2016-07-04 buy  sh000300
2016-

In [57]:
print ('total account = ', account)
print('total len = ', len(profits))
winning_len = len(list(filter(lambda x: x > 0, profits)))
print('winning len = ', winning_len)
print("winning rate = ", winning_len * 1.0 / len(profits))
print("avg profit = ", sum(profits) / len(profits))
print("max loss = ", min(profits))

total account =  1770129.979547011
total len =  334
winning len =  194
winning rate =  0.5808383233532934
avg profit =  2305.778381877278
max loss =  -94872.38172251871
